In [1]:
import train_nn

In [2]:
import random, os, numpy as np, tensorflow as tf
from config import *
from tf_graph import FlappyGraph

In [3]:
def add_jumps_to_training(training_images, last_jumps):
    print("Parsing data...")
    iter_counter = 0
    X_data = []
    # (game frames, height, width)
    for i, game in enumerate(training_images):
        X_data.append([])
        for j, image in enumerate(game):
            X_data[iter_counter].append(np.append(image.ravel(), last_jumps[i][j]))
        X_data[iter_counter] = np.array(X_data[iter_counter], dtype=np.float32)
        iter_counter += 1

    return np.array(X_data)

In [4]:
print("Loading data...")
training_images = np.load(os.path.join(DATA_DIR, "images.npy"))
actions = np.load(os.path.join(DATA_DIR, "actions.npy"))
rewards = np.load(os.path.join(DATA_DIR, "adjusted_rewards.npy"))
last_jumps = np.load(os.path.join(DATA_DIR, "last_jumps.npy"))
X_data = add_jumps_to_training(training_images = training_images, last_jumps = last_jumps)

Loading data...
Parsing data...


In [5]:
flappy_graph = FlappyGraph(int((CANVAS_WIDTH * IMG_SCALE_FACTOR) * round(CANVAS_HEIGHT * IMG_SCALE_FACTOR)) + 1, 2)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)    

In [6]:
for i in range(5):
    for j in range(5):
        _, train_loss = sess.run([flappy_graph.train_step, flappy_graph.loss], 
                    feed_dict={
                        flappy_graph.inputs: X_data[i], 
                        flappy_graph.actions: actions[i], 
                        flappy_graph.rewards: rewards[i], 
                        flappy_graph.lr: 0.001}
                    )
        print("loss", train_loss)

loss nan
loss 20786.2
loss 20641.8
loss 20484.8
loss 20306.8
loss -1.43017e+06
loss -1.77568e+06
loss -1.80671e+06
loss -1.82892e+06
loss -1.8466e+06
loss 13714.6
loss 13713.9
loss 13713.2
loss 13712.4
loss 13711.7
loss 26134.2
loss 26131.6
loss 26128.9
loss 26126.3
loss 26123.7
loss -916.88
loss -916.884
loss -916.887
loss -916.89
loss -916.893


In [7]:
saver = tf.train.Saver()

if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)
    saver.save(sess, os.path.join(MODEL_DIR, "trained_flappy"))